下記のリンクをクリックするとGoogle Colabで実行することが出来ます  
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/2Nike2/LangChainPractice/blob/main/01_Get_started/00_01_RAG_search_example.ipynb)

### (事前準備: OpenAI APIキーの設定)
OpenAI APIを使う為のAPIキーを設定します  
このAPIキーについては、OpenAIのサイトで取得することが出来ます  
https://platform.openai.com/api-keys  
APIキーについては公開しないように注意してください  

In [ ]:
import os

# ここにあなたのOpenAIのAPIキーを入力してください
openai_api_key = 'yourapikey'

# 環境変数にAPIキーがまだ設定されていないならばAPIキーを設定
if os.getenv('OPENAI_API_KEY') is None:
    os.environ['OPENAI_API_KEY'] = openai_api_key


## RAG
検索強化生成の方法を確認します。

### ライブラリのインストール
LangChainのライブラリをインストールします

In [ ]:
!pip install langchain==0.1.4
!pip install langchain-openai==0.0.5
!pip install docarray==0.32.1
!pip install tiktoken=0.5.2


### 埋め込みモデル
文章を意味を凝縮した形で保存できるベクトルに変換する埋め込みモデルを用意します。  


In [ ]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings()


### ベクトルストア、リトリーバの作成
文章をベクトル化して保存するデータストア(ベクトルストア)  
及びそれを対象として検索を行うためのリトリーバを作成します。


In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_texts(
    ['太郎は区役所で働いています。', 'クマは蜂蜜を食べるのが好きです。'],
    embedding=embedding
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})


単独で実行すると入力した質問に関係のある文章が返ってきます。

In [ ]:
# 上位1件の文章を返す。
retriever.invoke('太郎はどこで働いていますか？')

### テンプレート
テンプレートを作成します。

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template('''\
下記の文脈に基づいて質問に答えて下さい:
{context}
質問:{question}
''')


## モデル
モデルを用意します。  

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4')


## 出力パーサ
出力を整えるパーサを用意します。  

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()


### 質問と文脈用意の並列処理
質問を文脈を後の処理に同時に渡せるようにしています。  
質問に関連する文脈を渡しつつ、同時に元の質問を後の処理に受け渡しています。  

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup_and_retrieval = RunnableParallel(
    {'context': retriever, 'question': RunnablePassthrough()}
)


## チェインの作成、実行
上記で用意した各種要素をつなげて実行します。

In [ ]:
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke('太郎はどこで働いていますか？')
